# Planes de pensiones y ALM

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from pandas.core.common import SettingWithCopyWarning
import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [26]:
def prestacion_definida(df, fecha_valoracion = '31/12/2021',  rvs = .015, rvp = .01,  h=12, margen_sol = .02,
                        tipo_int= .0022, beta=.15, edad_jubilacion = 65):
    def lx_participes():

        raw1 = 'https://raw.githubusercontent.com/Joevalencia/Planes_pensione_ALM/main/Participes_aportacion.csv'
        data1 = pd.read_csv(raw1).rename(columns = {'Antoni': 'Joan'})
        raw2 = 'https://raw.githubusercontent.com/Joevalencia/Planes_pensione_ALM/main/approximation_planex.csv'
        data2 = pd.read_csv(raw2).rename(columns = {'Antoni': 'Joan'})
        del data1['Unnamed: 0']
        del data2['Unnamed: 0']
        return data1, data2

    data1 = lx_participes()[0]
    data2 = lx_participes()[1]
    year = int(fecha_valoracion.split('/')[2])
#     l1, l2 = [], []
#     l3, l4 = [], []
#     l5 = []
#     participe = int(input('Ingresar el nùmero de participes del plan: '))
#     for i in range(0, participe):
        
#         p1 = str(input(f'Poner el nombre del participe del Plan numero {i+1}: '))
#         p2 = str(input(f'Ingresar la fecha de nacimiento {p1}: '))
#         p3 = str(input(f'Ingresar la fecha de alta en la empresa de {p1}: '))
#         p4 = float(input(f'Ingresar el salario pensionable 2021 de {p1}: '))
#         p5 = str(input(f'Ingresar el Sexo de {p1}: '))
#         print('='*55)
#         l1.append(p1)
#         l2.append(p2)
#         l3.append(p3)
#         l4.append(p4)
#         l5.append(p5)

    array1 = []
    for i in range(0, len(df['Participe'])):
        array1.append(int(df['Nacimiento'][i].split('/')[2]))

    array2 = []
    for i in range(0, len(df['Participe'])):
        array2.append(int(df['Alta'][i].split('/')[2]))
    an1 = [46, 38, 63, 30, 30, 24] ## edad actuarial  # AAA poner manualmente poner esto
    an2 = [25, 37, 29, 26, 20, 23] ## edad entradad actuarial  # AAA poner manualmente esto
    edad = [x-y for(x, y) in zip(array2, array1)] ## edad actuarial 2
    r1 = 1+rvs  ## Revalorizaciòn salarios
    r2 = 1+rvp  ## Revalorizaciòn pensiones 
    interes = (1+tipo_int)  ##interés
    t1 = np.arange(0, ((120-edad_jubilacion)*h))  ## mensilidad
    interes3 = interes**-(t1/h)  ## fraccionamiento 1/h
    #interes3 = interes2**-t1  ## intereses mensuales
    def variation_capital():
        j = 'https://raw.githubusercontent.com/Joevalencia/Planes_pensione_ALM/main/variation6.csv'
        ok = pd.read_csv(j)
        k = 'https://raw.githubusercontent.com/Joevalencia/Planes_pensione_ALM/main/px_mens_participes.csv'
        ok1 = pd.read_csv(k)
        return ok, ok1
    hello = variation_capital()
    ok, ok1 = hello[0], hello[1]
#     df = pd.DataFrame({'Participe':pd.Series(l1), 'Sexo': pd.Series(l5),'Nacimiento':pd.Series(l2), 
#                        'Alta':pd.Series(l3), 'Salario 2021':pd.Series(l4), 
#                       'Edad actuarial':pd.Series(an1), 'Edad entrada':pd.Series(an2)})

    e1 = round(((pd.to_datetime(datos['Alta'],dayfirst=True) - pd.to_datetime(datos['Nacimiento'], dayfirst=True))/365.25)/np.timedelta64(1, 'D')) ## Edad E
    fec_val= []
    for i in range(0,6):
        fec_val.append(fecha_valoracion)
    fec_val = pd.Series(fec_val)
    act_edad = round(((pd.to_datetime(fec_val,dayfirst=True) - pd.to_datetime(datos['Nacimiento'], dayfirst=True))/365.25)/np.timedelta64(1, 'D')) ## Edad A
    e1 = [48, 40, 65, 32, 31, 26]
    act_edad = [25, 37, 28, 26, 20, 23]
    df = df.join(pd.Series(act_edad, name='Edad actuarial'))
    df = df.join(pd.Series(e1, name='Edad entrada'))
    jubi2 = pd.to_numeric(df['Nacimiento'].str.split('/', expand=True)[2])+edad_jubilacion
    mes = pd.to_numeric(df['Nacimiento'].str.split('/', expand=True)[1])
    df = df.join(pd.Series(jubi2, name='Jubilacion'))
    proyec = round(df['Salario 2021']*(r1)**(edad_jubilacion-1-df['Edad actuarial']), 2)
    pj = round(beta*proyec/h, 2)
    df['Proyectado'] = proyec
    df['Proyectado'][2] = datos['Salario 2021'][2]
    
    jubi3 = pd.to_numeric(df['Alta'].str.split('/', expand=True)[2])
    df['PJ corriente'] = pj
    df['PJ corriente'][2] = beta*df['Proyectado'][2]/h
    joan1 = ok['Joan_var']*((data2.Joan[edad_jubilacion*h+mes[0]+1:]/data2.Joan[edad_jubilacion*h+mes[0]]).reset_index(drop=True))
    pere1 = ok['Pere_var']*((data2.Pere[edad_jubilacion*h+mes[1]+1:]/data2.Pere[edad_jubilacion*h+mes[1]]).reset_index(drop=True))
    marta1 = ok['Marta_var']*((data2.Marta[edad_jubilacion*h+mes[2]+1:]/data2.Marta[edad_jubilacion*h+mes[2]]).reset_index(drop=True))
    claudia1 = ok['Claudia_var']*((data2.Claudia[edad_jubilacion*h+mes[3]+1:]/data2.Claudia[edad_jubilacion*h+mes[3]]).reset_index(drop=True))
    marc1 = ok['Marc_var']*((data2.Marc[edad_jubilacion*h+mes[4]+1:]/data2.Marc[edad_jubilacion*h+mes[4]]).reset_index(drop=True))
    maria1 = ok['Maria_var']*((data2.Maria[edad_jubilacion*h+mes[5]+1:]/data2.Maria[edad_jubilacion*h+mes[5]]).reset_index(drop=True))
    new_df = pd.DataFrame({'Joan_VAA':joan1*interes3, 'Pere_VAA':pere1*interes3, 'Marta_VAA':marta1*interes3,
                           'Claudia_VAA':claudia1*interes3, 'Marc_VAA':marc1*interes3, 'Maria_VAA':maria1*interes3})
    VAAs = pd.Series(new_df.sum().round(2).to_list(), name='VAAj')
    
    ## A la fecha actual 2021
    VAA21_joan = round(VAAs[0]*((1+tipo_int)**-(edad_jubilacion-df['Edad actuarial'][0]))*(data1['Joan'][edad_jubilacion]/data1['Joan'][df['Edad actuarial'][0]]), 2)
    VAA21_pere = round(VAAs[1]*((1+tipo_int)**-(edad_jubilacion-df['Edad actuarial'][1]))*(data1['Pere'][edad_jubilacion]/data1['Pere'][df['Edad actuarial'][1]]), 2)
    VAA21_marta = round(VAAs[2]*((1+tipo_int)**-(edad_jubilacion-df['Edad actuarial'][2]))*(data1['Marta'][edad_jubilacion]/data1['Marta'][df['Edad actuarial'][2]]), 2)
    VAA21_claudia = round(VAAs[3]*((1+tipo_int)**-(edad_jubilacion-df['Edad actuarial'][3]))*(data1['Claudia'][edad_jubilacion]/data1['Claudia'][df['Edad actuarial'][3]]), 2)
    VAA21_marc = round(VAAs[4]*((1+tipo_int)**-(edad_jubilacion-df['Edad actuarial'][4]))*(data1['Marc'][edad_jubilacion]/data1['Marc'][df['Edad actuarial'][4]]), 2)
    VAA21_maria = round(VAAs[5]*((1+tipo_int)**-(edad_jubilacion-df['Edad actuarial'][5]))*(data1['Maria'][edad_jubilacion]/data1['Maria'][df['Edad actuarial'][5]]), 2)
    vaas21 = pd.Series([VAA21_joan, VAA21_pere, VAA21_marta, VAA21_claudia, VAA21_marc, VAA21_maria])
    
    ## A la fecha de alta en la empresa
    VAAp_joan = round(VAAs[0]*((1+tipo_int)**-(jubi2[0]-jubi3[0]+1))*(data1['Joan'][edad_jubilacion]/data1['Joan'][df['Edad entrada'][0]]), 2)
    VAAp_pere = round(VAAs[1]*((1+tipo_int)**-(jubi2[1]-jubi3[1]+1))*(data1['Pere'][edad_jubilacion]/data1['Pere'][df['Edad entrada'][1]]), 2)
    VAAp_marta = round(VAAs[2]*((1+tipo_int)**-(jubi2[2]-jubi3[2]+1))*(data1['Marta'][edad_jubilacion]/data1['Marta'][df['Edad entrada'][2]]), 2)
    VAAp_claudia = round(VAAs[3]*((1+tipo_int)**-(jubi2[3]-jubi3[3]+1))*(data1['Claudia'][edad_jubilacion]/data1['Claudia'][df['Edad entrada'][3]]), 2)
    VAAp_marc = round(VAAs[4]*((1+tipo_int)**-(jubi2[4]-jubi3[4]+1))*(data1['Marc'][edad_jubilacion]/data1['Marc'][df['Edad entrada'][4]]), 2)
    VAAp_maria = round(VAAs[5]*((1+tipo_int)**-(jubi2[5]-jubi3[5]+1))*(data1['Maria'][edad_jubilacion]/data1['Maria'][df['Edad entrada'][5]]), 2)
    vaas_p = pd.Series([VAAp_joan, VAAp_pere, VAAp_marta, VAAp_claudia, VAAp_marc, VAAp_maria])
    
    ## Coste normal Credito Unitario
    jo1 = vaas21[0]/(edad_jubilacion-df['Edad entrada'][0])
    jo2 = vaas21[1]/(edad_jubilacion-df['Edad entrada'][1])
    jo3 = vaas21[2]/(edad_jubilacion-df['Edad entrada'][2])
    jo4 = vaas21[3]/(edad_jubilacion-df['Edad entrada'][3])
    jo5 = vaas21[4]/(edad_jubilacion-df['Edad entrada'][4])
    jo6 = vaas21[5]/(edad_jubilacion-df['Edad entrada'][5])
    CN_credito = pd.Series([jo1, jo2, jo3, jo4, jo5, jo6])
    
    ## Valor actual actuarial servicios pasados
    VAASP_CU = pd.Series([  CN_credito[0]*(df['Edad actuarial'][0]-df['Edad entrada'][0]),
                            CN_credito[1]*(df['Edad actuarial'][1]-df['Edad entrada'][1]),
                            CN_credito[2]*(df['Edad actuarial'][2]-df['Edad entrada'][2]),
                            CN_credito[3]*(df['Edad actuarial'][3]-df['Edad entrada'][3]),
                            CN_credito[4]*(df['Edad actuarial'][4]-df['Edad entrada'][4]),
                            CN_credito[5]*(df['Edad actuarial'][5]-df['Edad entrada'][5])])
    ## Margen de solvencia 
    margen_CU = VAASP_CU*margen_sol
    
    ## dataframe Credito Unitario
    CU = pd.DataFrame({'VAAj':VAAs,'VAA 31/12/2021': vaas21, 
                       'CN 31/12/2021':CN_credito, 'VAASP 31/12/2021':VAASP_CU, 'MS 31/12/2021':margen_CU}).round(2)
    CU.index = df['Participe'].to_list()
    CU['VAASF 31/12/2021'] = CU.iloc[:,1]-CU.iloc[:,3]
    
    ## Edad de entrada -Coste Normal
    tiempo1 = np.arange(0, (edad_jubilacion-df['Edad entrada'][0]))
    #jo_1 = 
    jo_1 = vaas_p[0]/sum(((1+rvs)**tiempo1*(1+tipo_int)**-(tiempo1))*(data1['Joan'][df['Edad entrada'][0]+tiempo1]/data1['Joan'][df['Edad entrada'][0]]))
    tiempo2 = np.arange(0, (edad_jubilacion-df['Edad entrada'][1]))
    #jo_2 = 
    jo_2 = vaas_p[1]/sum(((1+rvs)**tiempo2*(1+tipo_int)**-(tiempo2))*(data1['Pere'][df['Edad entrada'][1]+tiempo2]/data1['Pere'][df['Edad entrada'][1]]))
    tiempo3 = np.arange(0, (edad_jubilacion-df['Edad entrada'][2]))
    #jo_3 = 
    jo_3= vaas_p[2]/sum(((1+rvs)**tiempo3*(1+tipo_int)**-(tiempo3))*(data1['Marta'][df['Edad entrada'][2]+tiempo3]/data1['Claudia'][df['Edad entrada'][2]]))
    tiempo4 = np.arange(0, (edad_jubilacion-df['Edad entrada'][3]))
    #jo_4 = 
    jo_4 = vaas_p[3]/sum(((1+rvs)**tiempo4*(1+tipo_int)**-(tiempo4))*(data1['Claudia'][df['Edad entrada'][3]+tiempo4]/data1['Marta'][df['Edad entrada'][3]]))
    tiempo5 = np.arange(0, (edad_jubilacion-df['Edad entrada'][4]))
    #jo_5 = 
    jo_5 = vaas_p[4]/sum(((1+rvs)**tiempo5*(1+tipo_int)**-(tiempo5))*(data1['Marc'][df['Edad entrada'][4]+tiempo5]/data1['Marc'][df['Edad entrada'][4]]))
    tiempo6 = np.arange(0, (edad_jubilacion-df['Edad entrada'][5]))
    #jo_6 = 
    jo_6 = vaas_p[5]/sum(((1+rvs)**tiempo6*(1+tipo_int)**-(tiempo6))*(data1['Maria'][df['Edad entrada'][5]+tiempo6]/data1['Maria'][df['Edad entrada'][5]]))
    CN_EE = pd.Series([jo_1,jo_2,jo_3,jo_4,jo_5,jo_6 ]).round(2)  ## Coste Normal Edad de entrada en el 0
    
    # Coste Normal en el 2021 de los participes
    p1 = CN_EE[0]*(1+tipo_int)**(df['Edad actuarial'][0]-df['Edad entrada'][0])
    p2 = CN_EE[1]*(1+tipo_int)**(df['Edad actuarial'][1]-df['Edad entrada'][1])
    p3 = CN_EE[2]*(1+tipo_int)**(df['Edad actuarial'][2]-df['Edad entrada'][2])
    p4 = CN_EE[3]*(1+tipo_int)**(df['Edad actuarial'][3]-df['Edad entrada'][3])
    p5 = CN_EE[4]*(1+tipo_int)**(df['Edad actuarial'][4]-df['Edad entrada'][4])
    p6 = CN_EE[5]*(1+tipo_int)**(df['Edad actuarial'][5]-df['Edad entrada'][5])
    
    CN_EE21 = pd.Series([p1,p2, p3, p4, p5, p6]).round(2)
    
    ## Valor actual actuarial servicios pasados edad de entradad al 2021
    w1 = np.arange(0, (edad_jubilacion-df['Edad actuarial'][0]))
    joa1 = round(sum(CN_EE21[0]*(1+rvs)**w1*(1+tipo_int)**-w1*(data1['Joan'][df['Edad actuarial'][0]+w1]/data1['Joan'][df['Edad actuarial'][0]])), 2)
    
    w1 = np.arange(0, (edad_jubilacion-df['Edad actuarial'][1]))
    joa2 = round(sum(CN_EE21[0]*(1+rvs)**w1*(1+tipo_int)**-w1*(data1['Joan'][df['Edad actuarial'][1]+w1]/data1['Joan'][df['Edad actuarial'][1]])), 2)
    
    w1 = np.arange(0, (edad_jubilacion-df['Edad actuarial'][2]))
    joa3 = round(sum(CN_EE21[0]*(1+rvs)**w1*(1+tipo_int)**-w1*(data1['Joan'][df['Edad actuarial'][2]+w1]/data1['Joan'][df['Edad actuarial'][2]])), 2)
    
    w1 = np.arange(0, (edad_jubilacion-df['Edad actuarial'][3]))
    joa4 = round(sum(CN_EE21[0]*(1+rvs)**w1*(1+tipo_int)**-w1*(data1['Joan'][df['Edad actuarial'][3]+w1]/data1['Joan'][df['Edad actuarial'][3]])), 2)
    
    w1 = np.arange(0, (edad_jubilacion-df['Edad actuarial'][4]))
    joa5 = round(sum(CN_EE21[0]*(1+rvs)**w1*(1+tipo_int)**-w1*(data1['Joan'][df['Edad actuarial'][4]+w1]/data1['Joan'][df['Edad actuarial'][4]])), 2)
    
    w1 = np.arange(0, (edad_jubilacion-df['Edad actuarial'][5]))
    joa6 = round(sum(CN_EE21[0]*(1+rvs)**w1*(1+tipo_int)**-w1*(data1['Joan'][df['Edad actuarial'][5]+w1]/data1['Joan'][df['Edad actuarial'][5]])), 2)
    
    serie_vaasp = vaas21-pd.Series([joa1, joa2, joa3, joa4, joa5, joa6])
    
    ## Margen metodo de entrada
    mar_EE = serie_vaasp*margen_sol
    
    df_EE = pd.DataFrame({'VAAj':VAAs,'VAA 31/12/2021': vaas21, 'CN 31/12/2021':CN_EE21, 'VAASP 31/12/2021':serie_vaasp, 'VAA en ALTA':vaas_p,
                         'MS 31/12/2021':mar_EE.round(2)})
    df_EE['VAASF 31/12/2021'] = df_EE.iloc[:,1]-df_EE.iloc[:,3]
    print('='*75)
    print('Hipotesis')
    print(f'Interés técnico valoración: {tipo_int*100}%')
    print(f'Revalorizaciòn Salario: {rvs*100}%')
    print(f'Revalorización Pensión: {rvp*100}%')
    print(f'Beta Pension: {beta}')
    print('='*75)
    print(df.set_index('Participe').T)
    print('='*75)
    print('METODO: Credito Unitario Proyectado')
    #print(round(CU.T, 2))
    #print(CU)
    print('='*75)
    print('METODO: Edad de entrada')
    print(df_EE)

    #return df_EE, df.set_index('Participe').T

In [45]:
datos = pd.DataFrame({'Participe':['Joan', 'Pere', 'Marta', 'Claudia', 'Marc', 'Maria'], 
                     'Nacimiento':['06/07/1973', '03/12/1981', '06/03/1957','09/11/1989','10/07/1990',
                                  '11/09/1995'],
                     'Alta':['15/02/1998','08/03/2019','01/09/1985','09/10/2015','01/10/2010',
                            '08/08/2018'],
                     'Salario 2021':[29944, 37586, 60172, 36236, 47755, 35612]})
datos

,Participe,Nacimiento,Alta,Salario 2021
0,Joan,06/07/1973,15/02/1998,29944
1,Pere,03/12/1981,08/03/2019,37586
2,Marta,06/03/1957,01/09/1985,60172
3,Claudia,09/11/1989,09/10/2015,36236
4,Marc,10/07/1990,01/10/2010,47755
5,Maria,11/09/1995,08/08/2018,35612


In [26]:
round(((pd.to_datetime('31/12/2021') - pd.to_datetime('03/12/1981'))/365.25)/np.timedelta64(1, 'D'))

41

In [25]:
(( pd.to_datetime('31/12/2021', dayfirst=True)- pd.to_datetime('06/07/1973', dayfirst=True) )/365.25)/np.timedelta64(1, 'D')

48.48733744010951

In [17]:
(pd.to_datetime('31/12/2021')- pd.to_datetime('06/07/1973')).days/366.25

48.43412969283276

In [49]:
prestacion_definida(datos)

Hipotesis
Interés técnico valoración: 0.22%
Revalorizaciòn Salario: 1.5%
Revalorización Pensión: 1.0%
Beta Pension: 0.15
Participe             Joan        Pere       Marta     Claudia        Marc  \
Nacimiento      06/07/1973  03/12/1981  06/03/1957  09/11/1989  10/07/1990   
Alta            15/02/1998  08/03/2019  01/09/1985  09/10/2015  01/10/2010   
Salario 2021         29944       37586       60172       36236       47755   
Edad actuarial          25          37          28          26          20   
Edad entrada            48          40          65          32          31   
Jubilacion            2038        2046        2022        2054        2055   
Proyectado        53516.22    56183.56     60172.0    63804.29    91944.28   
PJ corriente        668.95      702.29      752.15      797.55      1149.3   

Participe            Maria  
Nacimiento      11/09/1995  
Alta            08/08/2018  
Salario 2021         35612  
Edad actuarial          23  
Edad entrada            26  
Ju

<ipython-input-48-b9f0795d6361>:114: RuntimeWarning: divide by zero encountered in double_scalars
  jo3 = vaas21[2]/(edad_jubilacion-df['Edad entrada'][2])
<ipython-input-48-b9f0795d6361>:145: RuntimeWarning: divide by zero encountered in double_scalars
  jo_3= vaas_p[2]/sum(((1+rvs)**tiempo3*(1+tipo_int)**-(tiempo3))*(data1['Marta'][df['Edad entrada'][2]+tiempo3]/data1['Claudia'][df['Edad entrada'][2]]))


In [30]:
raw1 = 'https://raw.githubusercontent.com/Joevalencia/Planes_pensione_ALM/main/Participes_aportacion.csv'
data1 = pd.read_csv(raw1).rename(columns = {'Antoni': 'Joan'})
raw2 = 'https://raw.githubusercontent.com/Joevalencia/Planes_pensione_ALM/main/approximation_planex.csv'
data2 = pd.read_csv(raw2).rename(columns = {'Antoni': 'Joan'})
del data1['Unnamed: 0']
del data2['Unnamed: 0']
j = 'https://raw.githubusercontent.com/Joevalencia/Planes_pensione_ALM/main/variation6.csv'
ok = pd.read_csv(j)
k = 'https://raw.githubusercontent.com/Joevalencia/Planes_pensione_ALM/main/px_mens_participes.csv'
ok1 = pd.read_csv(k)


In [10]:
prestacion_definida(datos)

Hipotesis
Interés técnico valoración: 0.22%
Revalorizaciòn Salario: 1.5%
Revalorización Pensión: 1.0%
Beta Pension: 0.15
Participe             Joan        Pere       Marta     Claudia        Marc  \
Nacimiento      06/07/1973  03/12/1981  06/03/1957  09/11/1989  10/07/1990   
Alta            15/02/1998  08/03/2019  01/09/1985  09/10/2015  01/10/2010   
Salario 2021         29944       37586       60172       36236       47755   
Edad actuarial        49.0        41.0        65.0        32.0        31.0   
Edad entrada          25.0        38.0        28.0        26.0        19.0   
Jubilacion            2038        2046        2022        2054        2055   
Proyectado        37436.95    52935.26     60172.0    58351.71    78054.55   
PJ corriente        467.96      661.69      752.15       729.4      975.68   

Participe            Maria  
Nacimiento      11/09/1995  
Alta            08/08/2018  
Salario 2021         35612  
Edad actuarial        26.0  
Edad entrada          23.0  
Ju

In [7]:
t = np.arange(0, (120-65)*12)


In [28]:
round(sum(ok['Joan_var']*1.0022**-((t+1)/12)*ok1['Joan']), 2)

188483.08

0      0.999621
1      0.999242
2      0.998863
3      0.998485
4      0.998106
         ...   
655    0.000000
656    0.000000
657    0.000000
658    0.000000
659    0.000000
Name: Joan, Length: 660, dtype: float64

In [24]:
round(data2['Joan'][781]/data2['Joan'][780],6)

0.999621

In [94]:
data1.Antoni[65]/data1.Antoni[49]

0.9590366479652812

In [6]:
126016/sum(1.01**np.arange(0, (47-19))*1.0125**-(np.arange(0, (47-19)))*data1['Antoni'][np.arange(19, 47)]/data1['Antoni'][19])

4680.726638398424

In [58]:
var = np.roll(np.round(783.82*1.015**np.floor(np.arange(0,660) / 12 ), 2),-8)
var[-8:] = var[-9]
var[0] = 0
var_m5 = pd.Series(var)


In [61]:
varias = pd.DataFrame({'Joan': var_m0, 'Pere':var_m1, 'Marta':var_m2, 'Claudia':var_m3, 'Marc':var_m4, 'Maria':var_m5})
varias

,Joan,Pere,Marta,Claudia,Marc,Maria
0,0.00,0.00,0.00,0.00,0.00,0.00
1,467.96,671.62,752.15,729.40,975.68,783.82
2,467.96,671.62,752.15,740.34,975.68,783.82
3,467.96,671.62,752.15,740.34,975.68,783.82
4,467.96,671.62,752.15,740.34,975.68,795.58
...,...,...,...,...,...,...
655,1045.62,1478.50,1680.62,1629.79,2180.09,1751.39
656,1045.62,1478.50,1680.62,1629.79,2180.09,1751.39
657,1045.62,1478.50,1680.62,1629.79,2180.09,1751.39
658,1045.62,1478.50,1680.62,1629.79,2180.09,1751.39


In [51]:
dt1 = pd.DataFrame({'Joan_var':var_jo, 'Pere_var':var_pe, 'Marta_var':var_m1, 'Claudia_var':var_cl, 'Marc_var':var_m2,
                   'Maria_var':var_m3})
dt1.to_csv('variation6.csv')

In [28]:
var = np.roll(np.round(r2[0]*1.02**np.floor(np.arange(0,660) / 12 ), 2),-(r1[0]-1))
var[-(r1[0]-1):] = var[-(r1[0])]
var[0] = 0
var = pd.Series(var)
var

0         0.00
1       467.96
2       467.96
3       467.96
4       467.96
        ...   
655    1363.38
656    1363.38
657    1363.38
658    1363.38
659    1363.38
Length: 660, dtype: float64

In [40]:
var = np.roll(np.round(474.98*1.01**np.floor(np.arange(0,660) / 12 ), 2),-6)
var[-6:] = var[-7]
var[0] = 0
var = pd.Series(var)
var

0        0.00
1      474.98
2      474.98
3      474.98
4      474.98
        ...  
655    812.89
656    812.89
657    812.89
658    812.89
659    812.89
Length: 660, dtype: float64

In [28]:
var = np.roll(np.round(671.62*1.01**np.floor(np.arange(0,660) / 12 ), 2),-2)
var[-2:] = var[-3]
var[0] = 0
var1 = pd.Series(var)
var1

0         0.00
1       671.62
2       671.62
3       671.62
4       671.62
        ...   
655    1149.42
656    1149.42
657    1149.42
658    1149.42
659    1149.42
Length: 660, dtype: float64

In [41]:
new = pd.read_csv('variation6.csv')
new['Joan_var'] = var

In [45]:
new.to_csv('variation6.csv')

In [55]:
prestacion_definida(datos)

Hipotesis
Interés técnico valoración: 0.22%
Revalorizaciòn Salario: 1.5%
Revalorización Pensión: 1.0%
Beta Pension: 0.15
Participe             Joan        Pere       Marta     Claudia        Marc  \
Nacimiento      06/07/1973  03/12/1981  06/03/1957  09/11/1989  10/07/1990   
Alta            15/02/1998  08/03/2019  01/09/1985  09/10/2015  01/10/2010   
Salario 2021         29944       37586       60172       36236       47755   
Edad actuarial        49.0        41.0        65.0        32.0        31.0   
Edad entrada          25.0        38.0        28.0        26.0        19.0   
Jubilacion            2038        2046        2022        2054        2055   
Proyectado        37436.95    52935.26     60172.0    58351.71    78054.55   
PJ corriente        467.96      661.69      752.15       729.4      975.68   

Participe            Maria  
Nacimiento      11/09/1995  
Alta            08/08/2018  
Salario 2021         35612  
Edad actuarial        26.0  
Edad entrada          23.0  
Ju

In [42]:
prestacion_definida(datos)[0]

Hipotesis
Interés técnico valoración: 0.22%
Revalorizaciòn Salario: 1.5%
Revalorización Pensión: 1.0%
Beta Pension: 0.15
Credito Unitario


,Unnamed: 0,Joan_var,Pere_var,Marta_var,Claudia_var,Marc_var,Maria_var
0,0,0.00,0.00,0.00,0.00,0.00,0.00
1,1,467.96,671.62,752.15,729.40,975.68,783.82
2,2,467.96,671.62,752.15,740.34,975.68,783.82
3,3,467.96,671.62,752.15,740.34,975.68,783.82
4,4,467.96,671.62,752.15,740.34,975.68,795.58
...,...,...,...,...,...,...,...
655,655,1045.62,1478.50,1680.62,1629.79,2180.09,1751.39
656,656,1045.62,1478.50,1680.62,1629.79,2180.09,1751.39
657,657,1045.62,1478.50,1680.62,1629.79,2180.09,1751.39
658,658,1045.62,1478.50,1680.62,1629.79,2180.09,1751.39


In [35]:
ty1 = np.arange(0, (65-26))
sum((1.015**ty1)*(1.0022**-ty1)*(data1['Maria'][26+ty1]/data1['Maria'][26]))

    #*114830.79

49.87125853534136

In [24]:
ty2 = np.arange(0, 3)
sum(data1['Maria'][23+ty2]/data1['Maria'][23]*1.0022**-ty2*1.015**ty2)

3.038207441989004

In [25]:
data1['Maria'][65]/data1['Maria'][23]

0.981990017907161

In [23]:
fec_val= []
for i in range(0,6):
    fec_val.append('31/12/2021')
fec_val = pd.Series(fec_val)
round(((pd.to_datetime(fec_val) - pd.to_datetime(datos['Nacimiento']))/365.25)/np.timedelta64(1, 'D'))-1

0    48.0
1    40.0
2    64.0
3    31.0
4    30.0
5    25.0
dtype: float64

In [9]:
prestacion_definida(datos)

Hipotesis
Interés técnico valoración: 0.22%
Revalorizaciòn Salario: 1.5%
Revalorización Pensión: 1.0%
Beta Pension: 0.15
Credito Unitario
                       Joan       Pere      Marta    Claudia       Marc  \
VAAj              188517.60  278196.98  322348.83  357972.95  422803.25   
VAA 31/12/2021    174548.75  253391.48  322348.83  326394.01  379060.39   
Alta Firm         162838.79  250892.88  284399.95  321197.68  368209.42   
CN 31/12/2021       4363.72    9388.87    8712.13    8369.08    8238.68   
VAASP 31/12/2021  104729.28   28166.61  322348.81   50214.48   98864.16   
Margen              2094.59     563.33    6446.98    1004.29    1977.28   

                      Maria  
VAAj              390488.14  
VAA 31/12/2021    352052.75  
Alta Firm         348879.77  
CN 31/12/2021       8382.21  
VAASP 31/12/2021   25146.63  
Margen               502.93  
0    27.72
1     3.04
2    45.28
3     6.32
4    12.86
5     3.04
dtype: float64


# Planes de pensiones

### Aportaciò definida

In [102]:
t1 = np.arange(0, 660)  ## mensilidad
interes = 1.0125  # Interes (1+interes)
interes2 = interes**(1/12)  ## fraccionamiento 1/h
interes3 = interes2**-t1
len(interes3)  # factor financiero

660

In [3]:
raw1 = 'https://raw.githubusercontent.com/Joevalencia/Planes_pensione_ALM/main/Participes_aportacion.csv'
data1 = pd.read_csv(raw1)
raw2 = 'https://raw.githubusercontent.com/Joevalencia/Planes_pensione_ALM/main/approximation_planex.csv'
raw3 = 'https://raw.githubusercontent.com/Joevalencia/Planes_pensione_ALM/main//px_mens_participes.csv'
data2 = pd.read_csv(raw2)
data3 = pd.read_csv(raw3)
del data1['Unnamed: 0']
del data2['Unnamed: 0']

In [ ]:
data3.

In [25]:
(data2.Antoni[787:]/data2.Antoni[786]).reset_index(drop=True)*var_m3

0        0.000000
1      467.604635
2      467.426953
3      467.249271
4      467.071589
          ...    
655           NaN
656           NaN
657           NaN
658           NaN
659           NaN
Length: 660, dtype: float64

In [18]:
var_m3

0         0.00
1       467.96
2       467.96
3       467.96
4       467.96
        ...   
655    1045.62
656    1045.62
657    1045.62
658    1045.62
659    1045.62
Length: 660, dtype: float64

In [23]:
1.0022**-np.arange(0, 660)*var_m3*data2['Antoni'].iloc[786:]

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
1436   NaN
1437   NaN
1438   NaN
1439   NaN
1440   NaN
Length: 1315, dtype: float64

In [15]:
(pd.Series(data2['Antoni'][787:].to_list())/(pd.Series(data2['Antoni'][786:]).to_list())[0])

0      0.999620
1      0.999241
2      0.998861
3      0.998481
4      0.998102
         ...   
649    0.000000
650    0.000000
651    0.000000
652    0.000000
653    0.000000
Length: 654, dtype: float64

In [40]:
1.0022**-(np.arange(0, 660)/12)

array([1.        , 0.99981688, 0.9996338 , 0.99945076, 0.99926774,
       0.99908476, 0.99890181, 0.9987189 , 0.99853602, 0.99835317,
       0.99817036, 0.99798758, 0.99780483, 0.99762212, 0.99743944,
       0.99725679, 0.99707418, 0.9968916 , 0.99670905, 0.99652654,
       0.99634406, 0.99616161, 0.9959792 , 0.99579682, 0.99561448,
       0.99543217, 0.99524989, 0.99506764, 0.99488543, 0.99470325,
       0.99452111, 0.99433899, 0.99415691, 0.99397487, 0.99379286,
       0.99361088, 0.99342893, 0.99324702, 0.99306514, 0.9928833 ,
       0.99270149, 0.99251971, 0.99233796, 0.99215625, 0.99197457,
       0.99179293, 0.99161131, 0.99142973, 0.99124819, 0.99106668,
       0.9908852 , 0.99070375, 0.99052234, 0.99034096, 0.99015961,
       0.9899783 , 0.98979702, 0.98961577, 0.98943456, 0.98925338,
       0.98907223, 0.98889111, 0.98871003, 0.98852899, 0.98834797,
       0.98816699, 0.98798604, 0.98780513, 0.98762424, 0.98744339,
       0.98726258, 0.9870818 , 0.98690105, 0.98672033, 0.98653

## Verdadero VAAj

In [95]:
tem = np.arange(0, 660)
valor21 = (pd.Series(data2['Antoni'][787:].to_list())/(pd.Series(data2['Antoni'][786:]).to_list())[0]*var_m0*1.0022**-(tem/12)).dropna().sum().round(2)
valor21

184948.21

In [105]:
prestacion_definida(datos)

Hipotesis
Interés técnico valoración: 0.22%
Revalorizaciòn Salario: 1.5%
Revalorización Pensión: 1.0%
Beta Pension: 0.15
Credito Unitario
0      0.999620
1      0.999240
2      0.998860
3      0.998481
4      0.998101
         ...   
648    0.000000
649    0.000000
650    0.000000
651    0.000000
652    0.000000
Name: Joan, Length: 653, dtype: float64


In [100]:
pd.Series(data2['Antoni'][787:].to_list())/(pd.Series(data2['Antoni'][786:]).to_list())[0]

0      0.999620
1      0.999241
2      0.998861
3      0.998481
4      0.998102
         ...   
649    0.000000
650    0.000000
651    0.000000
652    0.000000
653    0.000000
Length: 654, dtype: float64

## Verdadero VAA al 31-12-2021

In [96]:
((1.0022**-(65-49)*(data1['Antoni'][65]/data1['Antoni'][49]))*valor21).round(2)

171243.84

## Verdadero CN 31-12-2021

In [9]:
round((1/(65-49))*171243.84, 2)

10702.74

## Valor Actual Actuarial Servicios Pasados

In [16]:
round(171243.84*((49-25)/(65-25)), 2)

102746.3

# Margen de solvencia

In [17]:
102746.3*.02

2054.926

## Edad de entrada

In [18]:
bo = np.arange(0, 40)
184948.21/sum(1.015**bo*1.0022**-bo*(data1.Antoni[25+bo]/data1.Antoni[25]))  ## Coste normal al 31-12-


3633.8759609102735

## Coste normal actual 2021

In [85]:
round(4010.9*1.015**24, 2)

5733.59

# Servicios pasados

In [89]:
bo2 = np.arange(0, 16)
sum(5733.59*1.015**bo2*1.0022**-bo2*(data1.Antoni[49+bo2]/data1.Antoni[49]))

99439.19579154288

In [92]:
round(118502.45-99441, 2)  ## Servicios pasados al 2021

19061.45

# Maria Capital de Cobertura

In [64]:
valor21 = (pd.Series(data2['Maria'][790:].to_list())/(pd.Series(data2['Maria'][789:]).to_list())[0]*var_m5*1.0022**-(np.arange(0, 660)/12)).dropna().sum().round(2)
valor21

380114.32

In [85]:
pd.Series(data2['Maria'][790:].to_list())/(pd.Series(data2['Maria'][789:]).to_list())[0]

0      0.999893
1      0.999785
2      0.999678
3      0.999569
4      0.999461
         ...   
646    0.000000
647    0.000000
648    0.000000
649    0.000000
650    0.000000
Length: 651, dtype: float64

# Valor actual actuarial al 2021

In [65]:
((1.0022**-(65-26)*(data1['Maria'][65]/data1['Maria'][26]))*valor21).round(2)

342700.02

In [66]:
round((1/(65-23))*342700.02, 2)

8159.52

## Servicios Pasados Metodo credito unitario

In [67]:
round(((26-23)/(65-23))*342700.02, 2)

24478.57

## Margen credito unitario

In [68]:
24478.57*.02

489.5714

# Edad de entrada
### Valor actual actuarial al 2018 cuando se dio de alta

In [69]:
((1.0022**-(65-23)*(data1['Maria'][65]/data1['Maria'][23]))*valor21).round(2)

340358.48

In [70]:
round((data1.Maria[65]/data1.Maria[23])*valor21*1.0022**-(65-23), 2)

340358.48

### Coste normal cuando entrò

In [71]:
bo = np.arange(0, (65-23))
340358.48/sum(1.015**bo*1.0022**-(bo/12)*(data1.Maria[23+bo]/data1.Maria[23]))  ## Coste normal al 31-12-2018 cuando se da de alta

5932.97535218501

## Coste normal al 2021

In [72]:
5932.98*1.015**3  ## Coste normal al 2021

6203.988885307497

## Servicios Futuros

In [74]:
bo2 = np.arange(0, 39)
sum(6203.99*1.015**bo2*1.0022**-bo2*(data1.Maria[26+bo2]/data1.Maria[26]))

309400.78924067237

### Servicios Pasados

In [76]:
round(342700.02-309400.79,2)

33299.23

In [87]:
round(33299.23*.02, 2)

665.98

# Servicios Pasados Maria

In [19]:
226878.22-214306.27  ## Servicios Pasados 

12571.950000000012

In [92]:
px = (ok1.iloc[65*12+1:]/ok1.iloc[65*12]).reset_index()  ## Matriz de px mensuales Participes
px.to_csv('px_mens_participes.csv')

In [ ]:
f = ['m', 'n']
for i in f:
    var = np.roll(np.round(b1['PJ corriente'][i]*1.01**np.floor(np.arange(0,660) / 12 ), 2),-6)
    var[-6:] = var[-7]
    var[0] = 0
    var = pd.Series(var)

In [61]:
var = np.roll(np.round(b1['PJ corriente'][5]*1.01**np.floor(np.arange(0,660) / 12 ), 2),-8)
var[-8:] = var[-9]
var[0] = 0
var_Maria = pd.Series(var)
var_Maria

0         0.00
1      1638.18
2      1638.18
3      1638.18
4      1654.56
        ...   
655    2803.60
656    2803.60
657    2803.60
658    2803.60
659    2803.60
Length: 660, dtype: float64

In [65]:
variation = pd.DataFrame({'Joan_var': var_Joan, 'Pere_var':var_Pere, 'Marta_var':var_Marta, 
                          'Claudia_var':var_Claudia, 'Marc_var':var_Marc,'Maria_var':var_Maria} )
variation

,Joan_var,Pere_var,Marta_var,Claudia_var,Marc_var,Maria_var
0,0.00,0.00,0.00,0.00,0.00,0.00
1,890.99,1323.46,1278.66,1480.15,1950.67,1638.18
2,890.99,1323.46,1278.66,1494.95,1950.67,1638.18
3,890.99,1323.46,1278.66,1494.95,1950.67,1638.18
4,890.99,1323.46,1278.66,1494.95,1950.67,1654.56
...,...,...,...,...,...,...
655,1524.85,2242.56,2188.31,2533.14,3338.40,2803.60
656,1524.85,2242.56,2188.31,2533.14,3338.40,2803.60
657,1524.85,2242.56,2188.31,2533.14,3338.40,2803.60
658,1524.85,2242.56,2188.31,2533.14,3338.40,2803.60


In [61]:
px.head()

,index,Antoni,Pere,Marta,Claudia,Marc,Maria
0,781,0.999621,0.999683,0.999737,0.999876,0.999740,0.999893
1,782,0.999242,0.999365,0.999474,0.999753,0.999480,0.999785
2,783,0.998863,0.999048,0.999212,0.999629,0.999220,0.999678
3,784,0.998485,0.998731,0.998949,0.999505,0.998961,0.999571
4,785,0.998106,0.998414,0.998686,0.999382,0.998701,0.999463


In [122]:
joan1 = ok['Joan_var']*ok1['Joan']
pere1 = ok['Pere_var']*ok1['Pere']
marta1 = ok['Marta_var']*ok1['Marta']
claudia1 = ok['Claudia_var']*ok1['Claudia']
marc1 = ok['Marc_var']*ok1['Marc']
maria1 = ok['Maria_var']*ok1['Maria']
new_df = pd.DataFrame({'Joan_VAA':joan1*interes3, 'Pere_VAA':pere1*interes3, 'Marta_VAA':marta1*interes3,
                       'Claudia_VAA':claudia1*interes3, 'Marc_VAA':marc1*interes3, 'Maria_VAA':maria1*interes3})
round(sum(new_df['Joan_VAA']), 2)

282648.57

In [128]:
VAAs = new_df.sum().round(2)
type(VAAs)

pandas.core.series.Series

In [140]:
VAAs

0    282648.57
1    430714.89
2    427609.83
3    554081.56
4    658762.27
5    621209.96
dtype: float64

In [137]:
var = np.roll(np.round(864.85*1.01**np.floor(np.arange(0,673) / 12 ), 2),-6)
var[-6:] = var[-7]
var[0] = 0
var = pd.Series(var)
var.tail(8)

665    1494.91
666    1509.86
667    1509.86
668    1509.86
669    1509.86
670    1509.86
671    1509.86
672    1509.86
dtype: float64